Méthodes d'ensemble
===================

**Author:** Laurent Siksous




# Table des matières







## Preamble



### Imports



In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_squared_log_error

import pickle

    # Out[2]:



### Functions



In [1]:
# Display all
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        display(df)

## Model Selection



### Load Data



In [1]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

display_all(train.describe(include='all').T)

In [1]:
X = train.drop('AvePrice', axis=1).to_numpy()
y = train['AvePrice'].to_numpy()

## Training



In [1]:
rf = RandomForestRegressor(random_state=42)

param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8]
}

gm = GridSearchCV(estimator=rf, param_grid=param_grid, cv= 5, oob_score=True, random_state=42)
gm.fit(X, y)
gm.best_params_

In [1]:
gm.score(X, y)
#gm.cv_results_

### Save model



In [1]:
with open('data/RF_california.pkl', 'wb') as file:
    pickle.dump(gm.best_estimator_, file)

## Bibliography



### References:PROPERTIES:




[california.bib](california.bib)

